### Exercicio 1 - Inteligencia Artificial



##### Adicionando dependencias e importações

In [14]:
import pandas as pd
import requests 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from requests.packages.urllib3.exceptions import InsecureRequestWarning


#### Coletando dados

In [15]:
url = "https://barramento.ifg.edu.br/ifg_barramento_ws/ws/252"
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

column_names = [
    "Campus",
    "Ano de Ingresso",
    "Período Letivo",
    "Curso",
    "Modalidade",
    "Presencial/Distância",
    "Sexo",
    "Nível",
    "Renda per Capita",
    "Etnia",
    "Ano de Nascimento",
    "Situação"
]

try:
    response = requests.get(url, verify=False)
    response.raise_for_status()

    data = response.json()

    processed_data = []
    for item in data:
        row = [sub_item["valor"] for sub_item in item["itens"]]
        processed_data.append(row)

    df = pd.DataFrame(processed_data, columns=column_names)

    output_path = 'data/data.csv'
    df.to_csv(output_path, index=False)
except requests.exceptions.RequestException as e:
    print(f"Erro na requisição: {e}")
except ValueError as e:
    print(f"Erro ao processar os dados: {e}")

#### Pré-processando os dados

In [19]:
def pre_process(df):
    """
    Trata valores nulos em um DataFrame automaticamente:
    - Numéricos: preenche com a média.
    - Categóricos: preenche com 'Desconhecido'.
    - Se a coluna tiver apenas dois valores únicos, substitui por 0 e 1.
    """
    for coluna in df.columns:
        if df[coluna].dtype in ["float64", "int64"]:  
            df[coluna] = df[coluna].fillna(df[coluna].mean())  
        elif df[coluna].dtype == "object":
            if df[coluna].nunique() == 2:
                valores = df[coluna].unique()
                df[coluna] = df[coluna].map({valores[0]: 0, valores[1]: 1}).fillna(df[coluna])
            else:
                df[coluna] = df[coluna].fillna("Desconhecido")  
    return df

df = pre_process(df)


#### Pós-processamento e transformação

In [20]:
def onehotencoder(df, colunas_categoricas):
    """
    Aplica OneHotEncoding às colunas categóricas especificadas no DataFrame.
    
    Parâmetros:
        df (pd.DataFrame): O DataFrame original.
        colunas_categoricas (list): Lista de nomes de colunas categóricas a serem transformadas.
        
    Retorno:
        pd.DataFrame: Um novo DataFrame com as colunas categóricas codificadas e as demais mantidas.
    """
    
    encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first")  
    
    encoded_arrays = encoder.fit_transform(df[colunas_categoricas])
    
    encoded_df = pd.DataFrame(
        encoded_arrays,
        columns=encoder.get_feature_names_out(colunas_categoricas),
        index=df.index
    )
    
    df = df.drop(columns=colunas_categoricas)
    df = pd.concat([df, encoded_df], axis=1)
    
    return df

colunas_categoricas = []
df = onehotencoder(df, colunas_categoricas)

#### Aplicação de modelos de regressão

In [ ]:
X = df.drop(columns=["Renda per Capita"])
y = df["Renda per Capita"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nAvaliação do Modelo:")
print(f"MSE: {mse:.2f}, R2: {r2:.2f}")

#### Plots e insights